In [9]:
import pandas as pd
import requests
import random
import json
import time
import glob
import os

In [4]:
root_path = '../Dataset'

raw_path = os.path.join(root_path, 'Raw')

if not os.path.exists(raw_path):
    os.makedirs(raw_path)

In [56]:
df = pd.read_csv(os.path.join(raw_path, 'SO.csv'), encoding='ISO-8859-1')
df.head()

,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_date,Question_favorite_count,Question_last_edit_date,Question_score,Question_tags,Question_view_count,...,Owner_location,Owner_reputation,Owner_up_votes,Owner_down_votes,Owner_views,Answer_body,Answer_comment_count,Answer_creation_date,Answer_last_edit_date,Answer_score
0,How do I merge SSIS package files?,<p>I am wondering if anyone has any advice on ...,5,0,2011-03-29 05:10:36.293000 UTC,1.0,2011-08-06 03:01:31.267000 UTC,10,version-control|merge|dvcs|ssis,12003,...,"Muncie, IN",950,183,7,125,<p>Using the free Visual Studio add-in <a href...,1.0,2011-08-30 16:29:00.047000 UTC,NaN,8.0
1,Is it good to commit files often if using Merc...,<p>It seems that it is suggested we can commit...,4,0,2010-06-10 18:47:32.023000 UTC,NaN,NaN,3,git|mercurial|dvcs,829,...,NaN,141372,1451,39,12992,<p>The best way to manage this (whether you ar...,4.0,2010-06-10 19:00:34.010000 UTC,NaN,11.0
2,How to drop a column from a Databricks Delta t...,<p>I have recently started discovering Databri...,7,1,2019-01-31 09:15:51.593000 UTC,2.0,2019-05-16 01:28:21.263000 UTC,16,sql|apache-spark|apache-spark-sql|databricks|d...,32123,...,NaN,2591,307,0,549,<p>There is no drop column option on Databrick...,4.0,2019-01-31 10:07:56.103000 UTC,NaN,16.0
3,What is the pyspark equivalent of MERGE INTO f...,"<p>The databricks <a href=""https://docs.databr...",1,2,2020-04-02 07:47:29.583000 UTC,2.0,NaN,5,merge|databricks|delta|delta-lake,7875,...,"Antwerpen, Belgium",655,302,9,89,<p>I managed to find the documentation using t...,2.0,2020-04-05 09:50:08.793000 UTC,NaN,9.0
4,Is a DVCS useful for one developer?,<p>Or would a conventional client-server VCS b...,6,0,2008-10-07 15:53:48.963000 UTC,5.0,2011-12-09 19:03:35.160000 UTC,16,version-control|dvcs,1018,...,"Rochester, NY, United States",112376,2947,204,15424,<p>Since you can still push to another machine...,1.0,2008-10-07 15:56:30.487000 UTC,NaN,15.0


In [57]:
tags = {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench', 'google-cloud-vertex-ai', 'databricks-unity-catalog',
        'databricks-ml', 'amazon-sagemaker', 'amazon-sagemaker-experiments', 'mlflow', 'dvc', 'wandb', 'clearml', 'comet-ml', 'neptune', 'pachyderm', 'lakefs'}

In [58]:
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))

In [59]:
arity = -1
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    arity = arity + 1
    print(f'{arity}: {post_number}')

0: 4551
1: 1716
2: 75
3: 5


In [6]:
filter_path = os.path.join(root_path, 'Filter')

if not os.path.isdir(filter_path):
    os.mkdir(filter_path)
    
round1_path = os.path.join(filter_path, 'Round#1')
    
if not os.path.isdir(round1_path):
    os.mkdir(round1_path)

In [78]:
df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid.to_csv(os.path.join(round1_path, 'SO_valid_posts.csv'),
                encoding='ISO-8859-1', index=False)

In [76]:
df_qualified = df_valid[df_valid['Question_score'] > -1]
df_qualified.to_csv(os.path.join(
    round1_path, 'SO_qualified_posts.csv'), encoding='ISO-8859-1', index=False)
df_qualified.groupby(['Question_valid_tags'])[
    'Question_valid_tags'].agg(['count'])

,count
Question_valid_tags,
['amazon-sagemaker'],764
"['azure-machine-learning-service', 'azure-machine-learning-studio']",11
['azure-machine-learning-service'],232
"['azure-machine-learning-studio', 'azure-machine-learning-service']",35
['azure-machine-learning-studio'],191
"['azure-machine-learning-workbench', 'azure-machine-learning-service']",1
"['azure-machine-learning-workbench', 'azure-machine-learning-studio', 'azure-machine-learning-service']",4
"['azure-machine-learning-workbench', 'azure-machine-learning-studio']",7
['azure-machine-learning-workbench'],3


In [79]:
df_qualified_completed = df_qualified.dropna(subset=['Answer_body'])
df_qualified_completed.to_csv(os.path.join(
    round1_path, 'SO_qualified_completed_posts.csv'), encoding='ISO-8859-1', index=False)
df_qualified_completed.groupby(['Question_valid_tags']).agg(['count'])
df_qualified_completed.groupby(['Question_valid_tags'])[
    'Question_valid_tags'].agg(['count'])

,count
Question_valid_tags,
['amazon-sagemaker'],28
['azure-machine-learning-service'],7
"['azure-machine-learning-studio', 'azure-machine-learning-service']",1
['azure-machine-learning-studio'],8
['dvc'],3
['google-cloud-vertex-ai'],1
['mlflow'],8


In [5]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(raw_path, 'DVC.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [7]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(raw_path, 'Weights & Biases.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [8]:
page = -1
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(raw_path, 'Guild AI.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_date'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(raw_path, 'SigOpt.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
external_post_number = {}
for file_name in glob.glob(os.path.join(raw_path, '*.json')):
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        tool_name = file_name.split('\\')[-1].split('.')[0]
        external_post_number[tool_name] = len(json_data)
external_post_number

{'Amazon SageMaker': 528,
 'Azure Machine Learning': 1435,
 'Databricks': 154,
 'Domino': 13,
 'DVC': 315,
 'Guild AI': 115,
 'MLFlow': 280,
 'Polyaxon': 43,
 'SigOpt': 15,
 'Vertex AI': 297,
 'Weights & Biases': 583}

In [7]:
sample_round1_path = os.path.join(round1_path, 'Sample')
    
if not os.path.isdir(sample_round1_path):
    os.mkdir(sample_round1_path)

In [11]:
# Randomly sample a fraction of elements from a list
def random_sample(l, frac):
    return random.sample(l, int(len(l) * frac))

fraction = 0.1
random.seed(0)

for file_name in glob.glob(os.path.join(raw_path, '*.json')):
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        tool_name = file_name.split('\\')[-1].split('.')[0]
        sample_posts = random_sample(json_data, fraction)
    json_sample_posts = json.dumps(sample_posts, indent='\t')
    with open(os.path.join(sample_round1_path, f'{tool_name}.json'), 'w') as outfile:
        outfile.write(json_sample_posts)